<a href="https://colab.research.google.com/github/bvm84/open_colab_ml/blob/master/Dog_vs_Cats_on_tensorflow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install tensorflow-gpu

In [73]:
import tensorflow as tf
print(tf.__version__)
print(tf.executing_eagerly())

2.0.0
True


In [0]:
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
from pathlib import PurePath
from pandas import DataFrame
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, Normalizer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
sys.path.append('/content/drive/My Drive/Colab/DvC/')
% matplotlib inline
np.random.seed(0)
clear_output()

In [75]:
!unzip -o /content/drive/My\ Drive/Colab/DvC/DvC.zip -d /content/DvC
clear_output()
print('Unzip done')

Unzip done


In [76]:
data_dir = pathlib.Path('/content/DvC')
train_ds = tf.data.Dataset.list_files(str(data_dir/'train/*/*'))
# train_ds = tf.data.Dataset.list_files(str(data_dir/'train/*/*'), shuffle=False)
val_ds = tf.data.Dataset.list_files(str(data_dir/'validation/*/*'))
image_count = len(list(train_ds))
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
CLASS_NAMES = np.array([item.name for item in data_dir.joinpath('train').glob('*') if item.name != "LICENSE.txt"])
print(CLASS_NAMES)
for f in train_ds.take(5):
  print(f.numpy())

['cats' 'dogs']
b'/content/DvC/train/cats/cat.794.jpg'
b'/content/DvC/train/dogs/dog.670.jpg'
b'/content/DvC/train/cats/cat.369.jpg'
b'/content/DvC/train/dogs/dog.908.jpg'
b'/content/DvC/train/dogs/dog.98.jpg'


In [0]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, '/')
    # The second to last is the class-directory
    return parts[-2] == CLASS_NAMES
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label


In [78]:
labeled_ds = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  [ True False]


In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    # Repeat forever
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

In [0]:
train_ds = prepare_for_training(labeled_ds)
image_batch, label_batch = next(iter(train_ds))

In [0]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
        plt.axis('off')
# show_batch(image_batch.numpy(), label_batch.numpy())

In [0]:
# define model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='linear')
    ])
model.compile(optimizer='adam', \
              loss='sparse_categorical_crossentropy', \
              metrics=['accuracy'])

In [87]:
model.fit_generator(train_ds, steps_per_epoch=STEPS_PER_EPOCH, epochs=2)

Epoch 1/2


ValueError: ignored